# Retrieving Road Network Data
Data about the road network, such as the locations of intersections, the length, lanes, speed limit and geometry of a road link, is an integral part in traffic simulation. Thinking of traffic modeling as getting outcomes (e.g., traffic flow) under given road supply and travel demand, the road network then defines the supply of the system.

In today's exercise, we will learn how to download raw road network data from OpenStreetMap (OSM). We will show three different ways to accomplish this task and you can select the most convenient method depending on the scale of your problem.
- Small and interactive: Overpass-turbo (graphical)
- Large datasets: Overpass API (shell commands)
- Cleaned data, good for network analysis: OSMnx (Python)

## 1. Small network: Overpass-turbo
Overpass-turbo is a handy way to download data for a small area. You can access overpass-turbo at https://overpass-turbo.eu/. Select an area of interest (e.g., UC Berkeley campus) and type your queries on the left.
<img src="https://github.com/UCB-CE170a/Fall2020/blob/master/python-exercises/Traffic%20Exercise%201/overpass-turbo-1-note.png?raw=1" alt="drawing" width="800"/>

Next, click "Run" and see the query results under the "Data" tab. You can copy and paste the result to a text editor for future use.

<table><tr>
<td> <img src="https://github.com/UCB-CE170a/Fall2020/blob/master/python-exercises/Traffic%20Exercise%201/overpass-turbo-2.png?raw=1" alt="Drawing" width="400"/>  </td>
<td> <img src="https://github.com/UCB-CE170a/Fall2020/blob/master/python-exercises/Traffic%20Exercise%201/overpass-turbo-3.png?raw=1" alt="Drawing" width="400"/> </td>
</tr></table>

## 2. Large network: Overpass API
If you want to download the road network data for a relatively large area (e.g., a city), it is better to use the [Overpass API from a command line terminal](http://overpass-api.de/command_line.html). Let's first create a file called `query.osm`, which specifies:
- output format: JSON.
- element type: roadways, including motorway, trunck, primary, secondary, tertiary, and residential roads.
- query area: within a polygon area defined by the lat/lon coordinates of the vertices.

In [ ]:
!echo "data=[out:json];way[highway~'motorway|motorway_link|motorway_junction|trunk|trunk_link|primary|primary_link|secondary|secondary_link|tertiary|tertiary_link|residential'](poly:'37.84615220228875 -122.4537615259554 37.9205424163833 -122.42627473534108 37.97662600406699 -122.27439451079097 37.93944332562459 -122.18153759427354 37.85457143622269 -122.14440323003141 37.78981170543388 -122.19181902134872 37.764909622499076 -122.34990028037001 37.78612226780035 -122.42013291803424 37.84615220228875 -122.4537615259554');(._;>;);out;"> query.osm

Next, we can download the data specified in `query.osm` into a file called `target.osm` from `https://overpass-api.de/api/interpreter`. `wget` is the Linux command for downloading content from web servers.

In [ ]:
!wget -O target.osm --post-file=query.osm "https://overpass-api.de/api/interpreter"

Now you should see the `target.osm` data in your Colab file directory. It is in JSON format that we have introduced briefly in [Python Exercise 5](https://github.com/UCB-CE170a/Fall2020/tree/master/python-exercises/Day%205). You can save the data and do further processing.

## 3. Cleaned network: OSMnx
Overpass-turbo and Overpass introduced above can be used to download raw data from the OSM. It is useful if you want to make customized road network from the scratch. However, this is often not the case as cleaning the raw data can be a time consuming process. For example, in traffic simulations, we need the road network as a directed graph, which the OSM raw data is close to, but not exactly so. If you want to get a cleaned graph, you can use the excellent [OSMnx package](https://geoffboeing.com/2016/11/osmnx-python-street-networks/). It was developed by [Geoff Boeing](https://geoffboeing.com/) while he was a PhD student at UC Berkeley. The [original blog post](https://geoffboeing.com/2016/11/osmnx-python-street-networks/) and [code repo](https://github.com/gboeing/osmnx) provide clear instructions of how to use it. Here we will show how to retrieve the road network of North Berkeley, the one we will use for the homework of this module.

In [ ]:
!sudo apt install libspatialindex-dev python-rtree
!pip install geopandas rtree osmnx

In [ ]:
import osmnx as ox
from shapely import geometry

In [ ]:
# you can get a cleaned road network by address
G0 = ox.graph_from_address('1878 Euclid Avenue, Berkeley, California', network_type='drive')

# plot the network
ox.plot_graph(G0)

In [ ]:
# create a shapely Polygon object. This polygon covers the corner of Berkeley to the northeast of Hearst and Martin Luther King Jr. Way
north_berkeley_shape = geometry.Polygon([[-122.25485626392467964, 37.90558645375407565], 
                                         [-122.27829721780133809, 37.90420130647954977], 
                                         [-122.27595312241368219, 37.87212981958465008], 
                                         [-122.2407916915986732, 37.87692456015033571],
                                        [-122.25485626392467964, 37.90558645375407565]])

# get road network by address
G = ox.graph_from_polygon(north_berkeley_shape, network_type='drive')

# plot the road network
ox.plot_graph(G)

In [ ]:
# convert the graph into nodes and edges Geopandas dataframe
north_berkeley_nodes, north_berkeley_edges = ox.graph_to_gdfs(G)
display(north_berkeley_nodes.head(2))
display(north_berkeley_edges.head(2))

This is how we retrieved the original network for the homework that is coming soon. The actual road network you will be using has been modified manually to remove some redundant features. However, without manual modifications, the outputs from OSMnx are in general good to be used directly for graph analysis and simulation.